In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
import numpy as np
import matplotlib.pyplot as plt
import feature_manager as fma
import classifier.multi_dnn_classifier as dnn
from random import randint
from keras import callbacks
from keras import losses
import visualizer
from keras.utils import np_utils
from sklearn.metrics import classification_report, confusion_matrix
import importlib
import utils


2023-01-20 15:36:01.682522: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-20 15:36:01.682666: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-20 15:36:01.682678: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-01-20 15:36:03.193701: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2023-01-20 15:36:03.193788: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnos

In [2]:
timeframe_in_ms = {"15m":15*60*1000,"1h":60*60*1000,"1d":24*60*60*1000}

symbol = "BTCUSDT"
trade_tf = "1d"
granular_tf = "1m"

data = pd.read_csv("../data/{}-{}.csv".format(
    symbol,trade_tf), 
    parse_dates=["Open Time"], 
    index_col = "Open Time"
)

granular_data = pd.read_csv("../nocommit/{}-{}.csv".format(
    symbol,granular_tf), 
    parse_dates=["Open Time"], 
    index_col = "Open Time"
)

In [3]:
tp = 0.1
sl = 0.07
md = 10

fm = fma.FeatureManager(data=data,target_col="trade_signal")

fm.prepare_trade_forward_data(
    granular_data=granular_data,
    take_profit_rate=tp,
    stop_loss_rate=sl,
    max_duration=md,
    timeframe_in_ms=timeframe_in_ms[trade_tf]
)

# loop_classifier(data=fm.df,cols=fm.cols,target_col="trade_signal",laps=3)

Processing timeframe 1/10
Processing timeframe 2/10
Processing timeframe 3/10
Processing timeframe 4/10
Processing timeframe 5/10
Processing timeframe 6/10
Processing timeframe 7/10
Processing timeframe 8/10
Processing timeframe 9/10
Processing timeframe 10/10
Completed. Value counts of target_col:
0    783
1    610
2    570
Name: trade_signal, dtype: int64


In [4]:
fm.build_features(lags=60)

Calculating features...

Adding feature: returns, dir, sma, boll, boll7, boll14, boll21, min, min7, min14, min21, max, max7, max14, max21, mom, mom7, mom14, mom21, vol, vol7, vol14, vol21, obv, mfi7, mfi14, mfi21, rsi7, rsi14, rsi21, adx7, adx14, adx21, roc, roc7, roc14, roc21, atr7, atr14, atr21, bop, ad, adosc, trange, ado, willr7, willr14, willr21, dx7, dx14, dx21, trix, ultosc, high, low, 

Normalizing feature: returns_lag_1, returns_lag_2, returns_lag_3, returns_lag_4, returns_lag_5, returns_lag_6, returns_lag_7, returns_lag_8, returns_lag_9, returns_lag_10, returns_lag_11, returns_lag_12, returns_lag_13, returns_lag_14, returns_lag_15, returns_lag_16, returns_lag_17, returns_lag_18, returns_lag_19, returns_lag_20, returns_lag_21, returns_lag_22, returns_lag_23, returns_lag_24, returns_lag_25, returns_lag_26, returns_lag_27, returns_lag_28, returns_lag_29, returns_lag_30, returns_lag_31, returns_lag_32, returns_lag_33, returns_lag_34, returns_lag_35, returns_lag_36, returns_lag_37

In [ ]:
importlib.reload(dnn)

In [ ]:
classifier = dnn.MultiClassDNNClassifer()

classifier.prepare_data(
    data = fm.df,
    cols = fm.cols,
    is_shuffle = True,
    y_to_categorical=True,
    rebalance = None,
    random_state=100,
    target_col="trade_signal"
)

initial_bias = utils.init_imbalanced_bias(
    data=classifier.data_train,
    target_col="trade_signal"
)

classifier.configure(
    hu = 200, 
    dropout=True, 
    input_dim=len(fm.cols),
    output_bias=initial_bias,
    class_num=3
)

In [5]:
dnn.loop_classifier(
    hu=2000,
    fm=fm,
    target_col="trade_signal",
    laps=10,
    batch_size=12,
    set_class_weight=False,
    save_check_point=False,
    early_stopping=True,
    gpu=False)


======= Lap 1 =======
Shuffling the data
Completed. Train: 1227, Validation: 263, Test: 264
Epoch 1/200


2023-01-20 15:38:07.875457: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1584000000 exceeds 10% of free system memory.


103/103 - 10s - loss: 1.5050 - tp: 565.0000 - fp: 551.0000 - tn: 1903.0000 - fn: 662.0000 - accuracy: 0.6705 - precision: 0.5063 - recall: 0.4605 - auc: 0.6762 - prc: 0.5018 - val_loss: 1.0353 - val_tp: 148.0000 - val_fp: 88.0000 - val_tn: 438.0000 - val_fn: 115.0000 - val_accuracy: 0.7427 - val_precision: 0.6271 - val_recall: 0.5627 - val_auc: 0.7779 - val_prc: 0.6363 - 10s/epoch - 94ms/step
Epoch 2/200


2023-01-20 15:38:16.006534: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1584000000 exceeds 10% of free system memory.


103/103 - 8s - loss: 0.7154 - tp: 868.0000 - fp: 287.0000 - tn: 2167.0000 - fn: 359.0000 - accuracy: 0.8245 - precision: 0.7515 - recall: 0.7074 - auc: 0.8789 - prc: 0.7955 - val_loss: 0.9724 - val_tp: 164.0000 - val_fp: 86.0000 - val_tn: 440.0000 - val_fn: 99.0000 - val_accuracy: 0.7655 - val_precision: 0.6560 - val_recall: 0.6236 - val_auc: 0.8077 - val_prc: 0.6830 - 8s/epoch - 78ms/step
Epoch 3/200


...

103/103 - 8s - loss: 0.5647 - tp: 957.0000 - fp: 217.0000 - tn: 2237.0000 - fn: 270.0000 - accuracy: 0.8677 - precision: 0.8152 - recall: 0.7800 - auc: 0.9220 - prc: 0.8576 - val_loss: 0.9839 - val_tp: 169.0000 - val_fp: 84.0000 - val_tn: 442.0000 - val_fn: 94.0000 - val_accuracy: 0.7744 - val_precision: 0.6680 - val_recall: 0.6426 - val_auc: 0.8320 - val_prc: 0.7211 - 8s/epoch - 80ms/step
Epoch 4/200


2023-01-20 15:38:32.415005: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1584000000 exceeds 10% of free system memory.


103/103 - 8s - loss: 0.4627 - tp: 1015.0000 - fp: 179.0000 - tn: 2275.0000 - fn: 212.0000 - accuracy: 0.8938 - precision: 0.8501 - recall: 0.8272 - auc: 0.9461 - prc: 0.9053 - val_loss: 1.1955 - val_tp: 163.0000 - val_fp: 88.0000 - val_tn: 438.0000 - val_fn: 100.0000 - val_accuracy: 0.7617 - val_precision: 0.6494 - val_recall: 0.6198 - val_auc: 0.8173 - val_prc: 0.7006 - 8s/epoch - 80ms/step
Epoch 5/200


2023-01-20 15:38:39.640713: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1584000000 exceeds 10% of free system memory.


103/103 - 7s - loss: 0.4140 - tp: 1041.0000 - fp: 156.0000 - tn: 2298.0000 - fn: 186.0000 - accuracy: 0.9071 - precision: 0.8697 - recall: 0.8484 - auc: 0.9566 - prc: 0.9191 - val_loss: 0.8405 - val_tp: 193.0000 - val_fp: 64.0000 - val_tn: 462.0000 - val_fn: 70.0000 - val_accuracy: 0.8302 - val_precision: 0.7510 - val_recall: 0.7338 - val_auc: 0.8725 - val_prc: 0.7958 - 7s/epoch - 69ms/step
Epoch 6/200
103/103 - 7s - loss: 0.2389 - tp: 1119.0000 - fp: 87.0000 - tn: 2367.0000 - fn: 108.0000 - accuracy: 0.9470 - precision: 0.9279 - recall: 0.9120 - auc: 0.9835 - prc: 0.9692 - val_loss: 0.9206 - val_tp: 181.0000 - val_fp: 75.0000 - val_tn: 451.0000 - val_fn: 82.0000 - val_accuracy: 0.8010 - val_precision: 0.7070 - val_recall: 0.6882 - val_auc: 0.8529 - val_prc: 0.7336 - 7s/epoch - 69ms/step
Epoch 7/200
103/103 - 7s - loss: 0.2037 - tp: 1132.0000 - fp: 74.0000 - tn: 2380.0000 - fn: 95.0000 - accuracy: 0.9541 - precision: 0.9386 - recall: 0.9226 - auc: 0.9883 - prc: 0.9787 - val_loss: 0.742